In [1]:
ls

01-sentence2vec.ipynb          __pycache__/       output/
02-reduction-dimensions.ipynb  grid_plot_umap.py


In [2]:
import umap
import pandas as pd 
import numpy as np
from grid_plot_umap import UmapOptimization

In [3]:
def generate_scatter_graph(plot_lst, dict_sort):
    entp_list = []
    n = 4
    datas = [dict_sort[i*n:(i+1)*n] for i in range(len(dict_sort)//n)]
    for batch in datas:
        fig, axes = plt.subplots(1, n, figsize=(40,10))
        for i, elm in enumerate(batch):
            idx = elm.get('index')            
            data_tmp = plot_lst[idx]

            # entropy 
            axes[i].contourf(data_tmp.get('xx'), data_tmp.get('yy'), data_tmp.get('grid_density'), cmap=plt.cm.Blues, alpha=0.7)
            # axes[i].scatter(data_tmp.get('x'), data_tmp.get('y'), alpha=0.9, s=0.1) 
            axes[i].scatter(data_tmp.get('x'), data_tmp.get('y'), alpha=0.9, s=20) 
            axes[i].set_title(f"UMAP {data_tmp.get('model_index')} Entropy = {round(data_tmp.get('entropy'), 4)} n_neighbors = {data_tmp.get('n_neighbors')} min_dist = {data_tmp.get('min_dist')} spread = {data_tmp.get('spread')}")
            
        plt.show()

In [4]:
df = pd.read_parquet('./output/imdb_test.parquet')
df

,text,label,vector
44057,I thought that this show was a blatant rip-off...,-1,"[-0.036628164, -0.09474771, 0.043206602, -0.01..."
16861,Is Def-Con 4 the worst movie ever made? It's v...,-1,"[-0.020052401, -0.081306525, -0.015959825, -0...."
4248,This is a complete rehash of the first film. I...,-1,"[0.020931583, -0.062211685, 0.014268616, -0.00..."
10982,"This is one of the best things on tv today, if...",-1,"[-0.05918846, 0.041175894, 0.028914733, -0.086..."
31692,The best movie of Czech genius Jiri Menzel (af...,-1,"[-0.087861955, 0.040946733, -0.09735404, -0.01..."
...,...,...,...
44812,... But not in a good way .<br /><br />I saw H...,-1,"[-0.016357826, -0.00880787, 0.023964314, -0.01..."
26098,"I personally, thought the ""The Howling IV"" was...",-1,"[-0.06197646, 0.050676696, 0.046477932, -0.013..."
46003,Any old time college movie has one plot: the b...,-1,"[-0.046979096, 0.0833949, 0.01569629, 0.005126..."
35711,Bigfoot movies can be fun. This one is terribl...,-1,"[-0.011460395, -0.0020965496, 0.0031189807, 0...."


In [5]:
vector = df['vector'].tolist()

In [7]:
# params = {'n_neighbors': [10,40,80,100],
#          'min_dist':    [0.01,0.1,0.2,0.4],
#          'spread':      [0.4,0.75,0.8,1,4]}

params = {'n_neighbors': [10,40,80],
          'min_dist':    [0.01,0.1,0.2],
          'spread':      [0.4,0.75,0.8]}
          
optmizer = UmapOptimization()
optmizer.input(params, np.array(vector))
result = optmizer.output()

  0%|          | 0/27 [00:00<?, ?it/s]

/usr/local/lib/python3.12/site-packages/umap/umap_.py:1394: RuntimeWarning: divide by zero encountered in power
  return 1.0 / (1.0 + a * x ** (2 * b))
/usr/local/lib/python3.12/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/usr/local/lib/python3.12/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/usr/local/lib/python3.12/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/usr/local/lib/python3.12/site-packages/umap/umap_

In [ ]:
rs_lst = []
for idx, model in enumerate(result) : 
    i = {}
    params = model.get_params()
    i['x'] = model.embedding_[:,0]
    i['y'] = model.embedding_[:,1]
    i['n_neighbors'] = params.get('n_neighbors')
    i['min_dist'] = params.get('min_dist')
    i['spread'] = params.get('spread')
    i['random_state'] = params.get('random_state')
    i['model_idx'] = idx
    i['model'] = model
    rs_lst.append(i)